In [4]:
import nltk
import os

nltk_data_path = "../data/libs/nltk_data"

os.makedirs(nltk_data_path, exist_ok=True)

nltk.data.path.append(nltk_data_path)

nltk.download('stopwords', download_dir=nltk_data_path)
# nltk.download('wordnet', download_dir=nltk_data_path)
# nltk.download('omw-1.4', download_dir=nltk_data_path)
# nltk.download('punkt_tab', download_dir=nltk_data_path)

print(f"NLTK Data Path: {nltk.data.path}")

[nltk_data] Downloading package stopwords to ../data/libs/nltk_data...


NLTK Data Path: ['/home/miellilas/nltk_data', '/home/miellilas/Documents/pba/googleplay_garminconnect/venv/nltk_data', '/home/miellilas/Documents/pba/googleplay_garminconnect/venv/share/nltk_data', '/home/miellilas/Documents/pba/googleplay_garminconnect/venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '../../data/libs/nltk_data', '../data/libs/nltk_data']


[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
import datetime

nlp = spacy.load('en_core_web_sm')

stop_words = set(stopwords.words('english'))

nltk_data_path = "../data/libs/nltk_data"
nltk.data.path.append(nltk_data_path)

In [7]:
garminconnect_df = pd.read_csv("../data/raw/20250313_1234_garminconnect_gplay_reviews.csv")
garminconnect_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ff93503a-df4d-48ca-b1bf-4021a67900cb,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Hate the new layout. Trying to figure out how ...,2,2,5.10,2025-03-12 07:03:48,NaN,NaN,5.10
1,6e147f11-76a6-4971-bb4a-41d0d2a2f8d0,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Love it, helps me tremendously",5,0,5.10,2025-03-12 06:42:08,NaN,NaN,5.10
2,004dafcd-a440-434d-a052-a96b935a39df,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Very Good and carefree,5,0,5.10,2025-03-12 05:05:14,NaN,NaN,5.10
3,f52f11f3-8c6e-41b3-a17c-963cbfbb2ad8,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Latest update wont allow me to create a workou...,2,0,5.10,2025-03-12 03:59:57,NaN,NaN,5.10
4,8ebbecf7-da22-4bdd-b43a-4729494212d9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Much better than fitbit, no issues syncing my ...",5,0,5.10,2025-03-12 02:37:58,NaN,NaN,5.10


In [10]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    text = text.lower()

    text = re.sub(r"http\S+|www\S+|@\w+|#\w+|\S+@\S+", "", text)

    text = re.sub(r"[^a-z\s]", "", text)

    doc = nlp(text)

    tokens = [
        token.lemma_ for token in doc
        if token.lemma_ not in stop_words
        and token.is_alpha
        and len(token.lemma_) > 2
    ]

    # 6. Return cleaned text
    return " ".join(tokens)


In [11]:
garminconnect_df['clean_content'] = garminconnect_df['content'].apply(preprocess_text)

garminconnect_df = garminconnect_df[garminconnect_df['clean_content'].str.strip().astype(bool)]

In [12]:
date_today = datetime.datetime.today().strftime("%Y%m%d_%H%M")
garminconnect_df.to_csv(f"../data/processed/{date_today}_garminconnect_df_cleaned.csv", index=False)

In [14]:
garminconnect_df[["content", "clean_content"]]

,content,clean_content
0,Hate the new layout. Trying to figure out how ...,hate new layout try figure load previous versi...
1,"Love it, helps me tremendously",love help tremendously
2,Very Good and carefree,good carefree
3,Latest update wont allow me to create a workou...,late update allow create workout category run ...
4,"Much better than fitbit, no issues syncing my ...",much well fitbit issue sync workout
...,...,...
125851,The edge 810 works well until I pair it with e...,edge work well pair either htc samsung nexus p...
125852,"As soon as I launch Connect on my phone, my 81...",soon launch connect phone turn happen connect ...
125853,Why no power data for cycling!!!!,power datum cycling
125854,I can't seem to send a prepared course (on the...,seem send prepared course garmin connect websi...
